## 1단계: 필수 패키지 설치

In [ ]:
!pip install --upgrade azure-ai-evaluation
!pip install pandas numpy matplotlib seaborn

## 2단계: 라이브러리 임포트 및 데이터 로드

In [1]:
import os
import json
import pandas as pd
import numpy as np
from typing import Dict, List, Any
import warnings
warnings.filterwarnings('ignore')

# Microsoft Foundry SDK - Retrieval Evaluators
try:
    from azure.ai.evaluation import RetrievalEvaluator, DocumentRetrievalEvaluator
    print("✓ Azure AI Evaluation 임포트 성공")
except ImportError:
    print("⚠ Azure AI Evaluation 설치 필요: pip install azure-ai-evaluation")

# 데이터 로드
with open('data/sample_qa.json', 'r', encoding='utf-8') as f:
    qa_data = json.load(f)

df = pd.DataFrame(qa_data)

print(f"\n✓ 데이터 로드 완료: {len(df)} 개 Q&A 쌍")
print(f"\n에이전트별 분포:")
print(df['agent'].value_counts())
print(f"\n✓ 각 행에는 question, response, context, ground_truth 필드가 포함되어 있습니다.")

✓ Azure AI Evaluation 임포트 성공

✓ 데이터 로드 완료: 34 개 Q&A 쌍

에이전트별 분포:
agent
product-agent          10
hallucination-agent     7
review-agent            6
policy-agent            6
rag-fail-agent          5
Name: count, dtype: int64

✓ 각 행에는 question, response, context, ground_truth 필드가 포함되어 있습니다.


## 3단계: RetrievalEvaluator 초기화

**RetrievalEvaluator**는 LLM을 사용하여 검색된 컨텍스트가 질문과 얼마나 관련이 있는지 평가합니다.
Ground Truth가 없어도 사용 가능합니다.

### 평가 기준
- 컨텍스트가 질문에 답하는데 얼마나 관련있는가?
- 가장 관련성 높은 정보가 상위에 위치하는가?
- 1-5점 Likert 척도 (높을수록 좋음)

In [ ]:
# ============================================================
# model_config 설정
# ============================================================
model_config = {
    "azure_endpoint": "https://your-endpoint.openai.azure.com/",
    "api_key": "your-api-key",
    "azure_deployment": "gpt-4",
    "api_version": "2024-12-01-preview"
}

# RetrievalEvaluator 초기화
try:
    retrieval_evaluator = RetrievalEvaluator(
        model_config=model_config,
        threshold=3  # 기본 threshold: 3점 이상이면 pass
    )
    
    print("✓ RetrievalEvaluator 초기화 성공")
    print("\n📋 출력 메트릭:")
    print("  - retrieval: 검색 관련성 점수 (1-5)")
    print("  - gpt_retrieval: GPT가 평가한 점수")
    print("  - retrieval_reason: 점수 근거 설명")
    print("  - retrieval_result: pass/fail (threshold 기준)")
    print("  - retrieval_threshold: 설정된 threshold 값")
except Exception as e:
    print(f"⚠ Evaluator 초기화 실패: {str(e)[:300]}")

✓ RetrievalEvaluator 초기화 성공

📋 출력 메트릭:
  - retrieval: 검색 관련성 점수 (1-5)
  - gpt_retrieval: GPT가 평가한 점수
  - retrieval_reason: 점수 근거 설명
  - retrieval_result: pass/fail (threshold 기준)
  - retrieval_threshold: 설정된 threshold 값


## 4단계: 전체 데이터셋에 대한 RetrievalEvaluator 실행

In [4]:
print("=== 전체 데이터 Retrieval 평가 시작 ===")
print(f"대상: {len(df)} 개 Q&A 쌍\n")

retrieval_results = []

for idx, row in df.iterrows():
    try:
        result = retrieval_evaluator(
            query=row['question'],
            context=row['context']
        )
        
        result['agent'] = row['agent']
        result['question'] = row['question']
        retrieval_results.append(result)
        
        if (idx + 1) % 5 == 0:
            print(f"  [{idx + 1}/{len(df)}] 완료")
            
    except Exception as e:
        print(f"  [{idx + 1}] 오류: {str(e)[:100]}")
        continue

retrieval_df = pd.DataFrame(retrieval_results)

print(f"\n{'='*60}")
print(f"✓ 평가 완료: {len(retrieval_results)}개")
print(f"{'='*60}")

# 전체 평균
print(f"\n📊 전체 평균 메트릭:")
print(f"  Retrieval 점수: {retrieval_df['retrieval'].mean():.2f}/5.0")
print(f"  Pass 비율: {(retrieval_df['retrieval_result'] == 'pass').sum() / len(retrieval_df) * 100:.1f}%")
print(f"  Fail 개수: {(retrieval_df['retrieval_result'] == 'fail').sum()}개")

# 점수 분포
print(f"\n📈 점수 분포:")
print(f"  최고 점수: {retrieval_df['retrieval'].max():.2f}/5.0")
print(f"  최저 점수: {retrieval_df['retrieval'].min():.2f}/5.0")
print(f"  표준편차: {retrieval_df['retrieval'].std():.2f}")

# 에이전트별 분석
print(f"\n🔍 에이전트별 상세 분석:")
for agent in sorted(retrieval_df['agent'].unique()):
    agent_data = retrieval_df[retrieval_df['agent'] == agent]
    pass_count = (agent_data['retrieval_result'] == 'pass').sum()
    print(f"\n  {agent}:")
    print(f"    개수: {len(agent_data)}개")
    print(f"    평균 점수: {agent_data['retrieval'].mean():.2f}/5.0")
    print(f"    Pass: {pass_count}개 ({pass_count/len(agent_data)*100:.1f}%)")
    print(f"    Fail: {len(agent_data)-pass_count}개")

# 최고 점수 샘플
print(f"\n🏆 최고 점수 3개 샘플:")
top3 = retrieval_df.nlargest(3, 'retrieval')[['agent', 'question', 'retrieval', 'retrieval_result']]
for idx, row in top3.iterrows():
    print(f"  {row['retrieval']:.2f}/5.0 ({row['retrieval_result']}) - [{row['agent']}]")
    print(f"    질문: {row['question'][:60]}...")

# 최저 점수 샘플
print(f"\n⚠️ 최저 점수 3개 샘플:")
bottom3 = retrieval_df.nsmallest(3, 'retrieval')[['agent', 'question', 'retrieval', 'retrieval_result', 'retrieval_reason']]
for idx, row in bottom3.iterrows():
    print(f"  {row['retrieval']:.2f}/5.0 ({row['retrieval_result']}) - [{row['agent']}]")
    print(f"    질문: {row['question'][:60]}...")
    print(f"    이유: {row['retrieval_reason'][:100]}...")

# 전체 결과 표시
print(f"\n\n{'='*60}")
print("📋 전체 평가 결과 데이터프레임")
print(f"{'='*60}\n")
from IPython.display import display
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 80)
display(retrieval_df[['agent', 'question', 'retrieval', 'retrieval_result', 'retrieval_reason']])

=== 전체 데이터 Retrieval 평가 시작 ===
대상: 34 개 Q&A 쌍

  [5/34] 완료
  [10/34] 완료
  [15/34] 완료
  [20/34] 완료
  [25/34] 완료
  [30/34] 완료

✓ 평가 완료: 34개

📊 전체 평균 메트릭:
  Retrieval 점수: 4.41/5.0
  Pass 비율: 85.3%
  Fail 개수: 5개

📈 점수 분포:
  최고 점수: 5.00/5.0
  최저 점수: 1.00/5.0
  표준편차: 1.44

🔍 에이전트별 상세 분석:

  hallucination-agent:
    개수: 7개
    평균 점수: 5.00/5.0
    Pass: 7개 (100.0%)
    Fail: 0개

  policy-agent:
    개수: 6개
    평균 점수: 5.00/5.0
    Pass: 6개 (100.0%)
    Fail: 0개

  product-agent:
    개수: 10개
    평균 점수: 5.00/5.0
    Pass: 10개 (100.0%)
    Fail: 0개

  rag-fail-agent:
    개수: 5개
    평균 점수: 1.00/5.0
    Pass: 0개 (0.0%)
    Fail: 5개

  review-agent:
    개수: 6개
    평균 점수: 5.00/5.0
    Pass: 6개 (100.0%)
    Fail: 0개

🏆 최고 점수 3개 샘플:
  5.00/5.0 (pass) - [product-agent]
    질문: 갤럭시 S24의 배터리 용량이 얼마나 되나요?...
  5.00/5.0 (pass) - [product-agent]
    질문: 노트북 A1 모델의 RAM은 업그레이드 가능한가요?...
  5.00/5.0 (pass) - [product-agent]
    질문: 무선 이어폰 X3의 배터리 수명은 어떻게 되나요?...

⚠️ 최저 점수 3개 샘플:
  1.00/5.0 (fail) - [rag-fail-agent

,agent,question,retrieval,retrieval_result,retrieval_reason
0,product-agent,갤럭시 S24의 배터리 용량이 얼마나 되나요?,5.0,pass,The context directly and fully answers the query about the Galaxy S24 batter...
1,product-agent,노트북 A1 모델의 RAM은 업그레이드 가능한가요?,5.0,pass,The context fully addresses the query about RAM upgradeability for the 노트북 A...
2,product-agent,무선 이어폰 X3의 배터리 수명은 어떻게 되나요?,5.0,pass,The context fully addresses the query about the battery life of the wireless...
3,product-agent,스마트워치 W2의 방수 등급은 어떻게 되나요?,5.0,pass,The context fully addresses the query by providing the exact waterproof rati...
4,product-agent,태블릿 T5의 화면 크기는?,5.0,pass,The context directly and clearly answers the query about the screen size of ...
5,product-agent,카메라 C1의 센서 크기는?,5.0,pass,"The context directly and clearly provides the sensor size of camera C1, full..."
6,product-agent,청소 로봇 R7의 먼지통 용량은?,5.0,pass,The context directly answers the query about the dustbin capacity of the R7 ...
7,review-agent,최근 구매한 노트북의 성능 리뷰를 보여주세요,5.0,pass,The context is highly relevant and directly addresses the query by summarizi...
8,review-agent,이어폰 X3 사용자들의 음질 평가는 어떤가요?,5.0,pass,The context directly and fully addresses the query about user sound quality ...
9,review-agent,스마트워치 사용자 리뷰 요약해줘,5.0,pass,The context directly summarizes smartwatch user reviews with relevant detail...
